<a href="https://colab.research.google.com/github/rahuldewangan05/BuildingGPT/blob/main/BigramModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### In this project we are making character based model which predicts the next character based on previous text

We are using tine shakespere dataset where we have all the poems written by shakespere in text document format


In [3]:
## Downloading tiny shakespere dataset
!wget https://raw.githubusercontent.com/Karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-29 19:33:31--  https://raw.githubusercontent.com/Karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-03-29 19:33:32 (27.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
## reading the text file
with open('input.txt','r',encoding='utf-8') as f:
  text=f.read()

In [5]:
print("length of dataset in characters: ",len(text))

length of dataset in characters:  1115394


In [6]:
## first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [7]:
## here are all the unique characters that occur in the text

## text -> sequece of char , set -> find unique char, list -> make list
chars=sorted(list(set(text)))

vocab_size=len(chars)

print("Unique character in this text are : ",''.join(chars))
print(vocab_size)

Unique character in this text are :  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## create a mapping from character to integer

In [8]:
## stoi is dictnary having elements as character:index
stoi = {ch:i for i,ch in enumerate(chars)}

## itos is dictnary having elements as index:character
itos = {i:ch for i,ch in enumerate(chars)}

## function to convert every character in string to it desired index acc to stoi and returns list
encode = lambda s: [stoi[i] for i in s]

## function to convert every index in list to it desired character and return a string
decode = lambda s: ''.join([itos[index] for index in s])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


## Encode the entire text of shakespeare poem and store it into torch.tensor

In [9]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print("length of data (encoded text file) :",len(data))
print("Shape of data (encoded text file) : ",data.shape)
print("dtatype of data (encoded text file) :",data.dtype)
print(data[:1000])

length of data (encoded text file) : 1115394
Shape of data (encoded text file) :  torch.Size([1115394])
dtatype of data (encoded text file) : torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0

## split the data into traning and testing

Note : we can't use train test split library because we have sequential data where sequence matters

In [10]:
## taking first 90% data for training and rest data for testing
n=int(0.9*len(data))
train_data=data[:n]
test_data=data[n:]

In [11]:
## defining block size to send to model for training
block_size=8

## defining sample training and testing data with their context and target integer
x=train_data[:block_size]
y=train_data[1:block_size+1]

print("sample training data ",x)
print("sample testing data ",y)

## printing how the dataset will look like
for t in range(block_size):
  context= x[:t+1]
  target= y[t]
  print("when context is ",context," the target is ",target)

sample training data  tensor([18, 47, 56, 57, 58,  1, 15, 47])
sample testing data  tensor([47, 56, 57, 58,  1, 15, 47, 58])
when context is  tensor([18])  the target is  tensor(47)
when context is  tensor([18, 47])  the target is  tensor(56)
when context is  tensor([18, 47, 56])  the target is  tensor(57)
when context is  tensor([18, 47, 56, 57])  the target is  tensor(58)
when context is  tensor([18, 47, 56, 57, 58])  the target is  tensor(1)
when context is  tensor([18, 47, 56, 57, 58,  1])  the target is  tensor(15)
when context is  tensor([18, 47, 56, 57, 58,  1, 15])  the target is  tensor(47)
when context is  tensor([18, 47, 56, 57, 58,  1, 15, 47])  the target is  tensor(58)


## Macking batches of input data

In [12]:
## setting seed so that random number so generated is same
torch.manual_seed(1337)
batch_size=4 ## we try to send 4 batches to model for training
block_size=8 ## define the context length

## function to genrate a small batch of data of inputs x and target y
def get_batch(split):
  data = train_data if split=="train" else test_data
  ix=torch.randint(len(data)-block_size,(batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('target')
print(yb.shape)
print(yb)

print("-------")

for b in range(batch_size):
  for t in range(block_size):
    context=xb[b,:t+1]    ## since in xb is stack so if would be list it would be similar to xb[b[:t+1]]
    target=yb[b,t]
    print("when input is ",context," the target is ",target)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------
when input is  tensor([24])  the target is  tensor(43)
when input is  tensor([24, 43])  the target is  tensor(58)
when input is  tensor([24, 43, 58])  the target is  tensor(5)
when input is  tensor([24, 43, 58,  5])  the target is  tensor(57)
when input is  tensor([24, 43, 58,  5, 57])  the target is  tensor(1)
when input is  tensor([24, 43, 58,  5, 57,  1])  the target is  tensor(46)
when input is  tensor([24, 43, 58,  5, 57,  1, 46])  the target is  tensor(43)
when input is  tensor([24, 43, 58,  5, 57,  1, 46, 43])  the target is  tensor(39)
when input is  tensor([44])  the target is  tensor(

## Using bigram language model

In [13]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):
    ## idx and target are both (B,T) tensor of integrs
    logists=self.token_embedding_table(idx) ## (B,T,C)

    if targets is None:
      loss=None
    else:
      B,T,C =logists.shape
      logists=logists.view(B*T,C)
      targets=targets.view(-1)
      loss=F.cross_entropy(logists,targets)

    return logists, loss

  def generate(self,idx,max_new_token):
    ## idx is (B,T) array of indices in the current context
    for _ in range(max_new_token):
      ## get the predictions
      logists, loss=self(idx)
      ## focus only on the last time step
      logists = logists[:,-1,:] ## becomes (B,C)
      probs=F.softmax(logists, dim=1) #(B,C)
      # sample from the distribution
      idx_next=torch.multinomial(probs,num_samples=1) # (B,1)
      ## append sampled index to the running sequence
      idx=torch.cat((idx,idx_next),dim=1) # (B,T+1)
    return idx


m = BigramLanguageModel(vocab_size)
logists,loss= m.forward(xb,yb)
print(logists.shape)
print(loss)

idx=torch.zeros((1,1),dtype=torch.long)
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_token=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


### Brief explaination about terms B,T,C
B -> batch size (here 32)

T -> time step or block size (here 8)

C -> channel or vocab size or size of embedding vector (here 65)

Note : here in forward fucntion of BigramModel the logits and target are flattend because we are using cross validation frunction for calculating loss so this cross validation function expect :

1. logits (input) as a 2d vector (N,C) where N->tokens and C-> no. of classes(here vocab size)
so we flatten logits from shape (B,T,C) to (B*T,C)

2. target as 1d vector (N) were N->token
so we flatten target from shape (B,T) to (B*T)

here B*T represent all the token in single line since B(Batch_Size) * T(block_size)

In [15]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [16]:
yb

tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

In [17]:
## Using AdmaW optimizer in pytorch here m.parameter()-> passes all the trainable parameter to optimiser AND lr -> learning rate
optimizer=torch.optim.AdamW(m.parameters(), lr=1e-3)

In [18]:
## note : the more we run this codeblock more the loss will be minimised

## minimising the loss
batch_size=32

## range is similar to epoch in tensorflow
for steps in range(1000):

  # sample a batch of data
  xb,yb=get_batch('train')

  #evaluate the loss by passing x_train and y_train to model
  logists,loss=m(xb,yb)
  # zero out the gradient
  optimizer.zero_grad(set_to_none=True)
  # compute gradient
  loss.backward()
  # update model parameter
  optimizer.step()

print(loss.item())

3.7218432426452637


In [19]:
## now after minimising the loss if again try model to print next character for the character embedded as zero -> it gives better result
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_token=100)[0].tolist()))


olylvLLko'TMyatyIoconxad.?-tNSqYPsx&bF.oiR;BD$dZBMZv'K f bRSmIKptRPly:AUC&$zLK,qUEy&Ay;ZxjKVhmrdagC-


Note  : print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_token=100)[0].tolist()))

this line means :
 - Starting with a blank/zero token
 - Asking the model "what character should come next?" 100 times
 - Converting the resulting sequence of token IDs back to readable text

### **Note**: till here we have build a very simple model which predicts what comes next based on the last character of context being provided to model

## Self attention blocks for procecessing tokens


In [20]:
## consider the following toy example

## setting seed to get same result everytime
torch.manual_seed(1337)
B,T,C = 4,8,2 ## batch, time_step, channel
x=torch.randn(B,T,C) ## random initialisation of tensor x with shape(B,T,C)
x.shape

torch.Size([4, 8, 2])

## note : here the concept we are using to getnext character is

for predicting the next character we just take average of all the charater appearing before the current character to predict the next character

In [21]:
## initialising a tensor named xbow filled dwith zeros of shape (B,T,C)
xbow=torch.zeros((B,T,C))

for b in range(B):
  for t in range(T):
    ## collecting the previous token from current token
    xprev=x[b,:t+1] ## (t,C)
    ## adding the average till the current term in xbow
    xbow[b,t]=torch.mean(xprev,0)

In [49]:
!git clone https://github.com/rahuldewangan05/BuildingGPT.git

Cloning into 'BuildingGPT'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [50]:
!git config --global user.email "dewanganr28@gmail.com"
!git config --global user.name "rahuldewangan05"

In [51]:
!git add .

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> BuildingGPT
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached BuildingGPT
hint: 
hint: See "git help submodule" for more information.


In [52]:
!git commit -m "BigramModel"

[main 6ec8d48] BigramModel
 1 file changed, 1 insertion(+)
 create mode 160000 BuildingGPT


In [53]:
!git push origin main

fatal: could not read Username for 'https://github.com': No such device or address
